In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,RobustScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv("../input/heart-disease-dataset/heart.csv")
df.head()
df.shape

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False,cbar=False)

In [ ]:
df.describe()

In [ ]:
categorical_cols = []
categorical_inds = []
counting_cols = []
counting_inds  = []
cnt=0
for i in df.columns:
    cnt+=1
    if df[i].nunique()<=5:
        categorical_cols.append(i)
        categorical_inds.append(cnt-1)
    

      
    else:
        counting_cols.append(i)
        counting_inds.append(cnt-1)

In [ ]:
print(categorical_cols)

In [ ]:
#They are already encoded

categorical_cols.remove('target')
categorical_cols.remove('sex')
categorical_inds.remove(1)
categorical_inds.remove(13)


print(categorical_cols)
print(counting_cols)

In [ ]:
for i in categorical_cols:
    print(i)
    print(df[i].value_counts())
    print()

In [ ]:
x = 1
plt.figure(figsize = (20,20))

for i in df.columns:
    plt.subplot(4,4,x)
    plt.boxplot(df[i])
    plt.title(i)
    x = x+1

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(df.drop(['target'],axis=1),
                                                df['target'],
                                                test_size=0.3,
                                                random_state=31)

In [ ]:
df.iloc[102,:-1].values

In [ ]:
from sklearn.preprocessing import RobustScaler
numerical_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())
])

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('ohe',OneHotEncoder(handle_unknown='ignore')),
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, counting_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],remainder='passthrough'
)

In [ ]:
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
])

In [ ]:
from sklearn import set_config
set_config(display='diagram')
pipe

In [ ]:
X_train = pipe.fit_transform(X_train)
X_test = pipe.transform(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix,accuracy_score


def eval_metric(model, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)
    
    print("Test_Set")
    print("confussion matrix")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print()
    print("Train_Set")
    print("confussion matrix")
    print(confusion_matrix(y_train, y_train_pred))
    print(classification_report(y_train, y_train_pred))
    print()
    dt_acc_score = accuracy_score(y_test, y_pred)
    print("Accuracy :",dt_acc_score*100,'\n')

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier1 = LogisticRegression(solver='lbfgs', max_iter=10000,random_state=2)
classifier1.fit(X_train, y_train)
y_pred = classifier1.predict(X_test)

In [ ]:
eval_metric(classifier1, X_train, y_train, X_test, y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier2 = DecisionTreeClassifier(criterion = 'entropy', random_state = 2)
classifier2.fit(X_train, y_train)
y_pred = classifier2.predict(X_test)

In [ ]:
eval_metric(classifier2, X_train, y_train, X_test, y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'max_depth': [3,4,5,6,7,8,9,10,11,12,13],
    'min_samples_leaf': [10,20, 25, 75, 50, 100,150,200],
    'criterion': ["gini", "entropy"],
    'splitter':['best','random'],
}          
grid_search = GridSearchCV(estimator = classifier2,
                           param_grid = params,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
best_model = grid_search.best_estimator_

print(best_accuracy)
print(best_parameters)
print(best_model)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier3 = RandomForestClassifier()
classifier3.fit(X_train, y_train)
y_pred = classifier3.predict(X_test)

In [ ]:
eval_metric(classifier3, X_train, y_train, X_test, y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'n_estimators':[90,100,120,130],
    'max_depth': [3,4,5,6,7,8,9,10,11,12,13],
    'min_samples_leaf': [20, 25, 75],
    'criterion': ["gini", "entropy"],
}          
rf_grid_search = GridSearchCV(estimator = classifier3,
                           param_grid = params,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
rf_grid_search.fit(X_train, y_train)
best_accuracy = rf_grid_search.best_score_
best_parameters = rf_grid_search.best_params_
best_model = rf_grid_search.best_estimator_

print(best_accuracy)
print(best_parameters)
print(best_model)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier4 = KNeighborsClassifier(n_neighbors = 2)
classifier4.fit(X_train, y_train)
y_pred = classifier4.predict(X_test)

In [ ]:
eval_metric(classifier4, X_train, y_train, X_test, y_test)

In [ ]:
scores=[]

for i in range(1,40):
    classifier4 = KNeighborsClassifier(n_neighbors = i)
    classifier4.fit(X_train, y_train)
    y_pred = classifier4.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

In [ ]:
maxi_ind=-1
maxi=0
cnt=0
for i in scores:
    cnt+=1
    if maxi<i:
        max_ind = cnt
        maxi = i
print(max_ind)

In [ ]:
classifier4 = KNeighborsClassifier(n_neighbors = max_ind)
classifier4.fit(X_train, y_train)
y_pred = classifier4.predict(X_test)

In [ ]:
eval_metric(classifier4, X_train, y_train, X_test, y_test)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(df.drop(['target'],axis=1),
                                                df['target'],
                                                test_size=0.3,
                                                random_state=31)
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [ ]:
#Numerical

numerical_transformer = Pipeline(steps=[
  
    ('scaler', RobustScaler())
])

In [ ]:
#Categorical

categorical_transformer = Pipeline(steps=[

    ('ohe',OneHotEncoder(handle_unknown='ignore')),

])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, counting_inds),
        ('cat', categorical_transformer, categorical_inds)
    ],remainder='passthrough'
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()

In [ ]:

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', classifier)
])

pipe.fit(X_train,y_train)

In [ ]:
y_pred = pipe.predict(X_test)
y_pred

In [ ]:
eval_metric(pipe, X_train, y_train, X_test, y_test)

In [ ]:
import pickle

pickle.dump(pipe,open('pipe.pkl','wb'))

In [ ]:
pipe = pickle.load(open('pipe.pkl','rb'))

In [ ]:
test1 = np.array([53,1,0,140,203,1,0,155,1,3.1,0,0,3]).reshape(1,13)

In [ ]:
test2 = np.array([45,1,2,255,255,1,2,255,1,1.4,2,4,3]).reshape(1,13)

In [ ]:
pipe.predict(test1)

In [ ]:
df.iloc[102,:].values

In [ ]:
pipe.predict(df.iloc[102,:-1].values.reshape(1,13))